# Particle Filter for TurtleBot3: Monte Carlo Localization

This notebook demonstrates **Monte Carlo Localization (MCL)** using a Particle Filter for the TurtleBot3 robot.

**Problem**: Global localization - determine robot pose in a known map without prior position ("kidnapped robot")

**State**: $[x, y, \theta]$ (2D pose)

**Measurements**: LIDAR scans

**Challenge**: Global localization requires multi-modal distribution (robot could be in many locations)

**Reference**: Gordon et al. (1993), "Novel approach to nonlinear/non-Gaussian Bayesian state estimation"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pykal import DynamicalSystem, ROSNode
from pykal.algorithm_library.estimators.pf import PF
from pykal.gazebo import start_gazebo, stop_gazebo

np.random.seed(42)

## 1. Launch Gazebo Simulation

Start TurtleBot3 in a known environment.

In [ ]:
# Start Gazebo with TurtleBot3 in turtlebot3_world
gz = start_gazebo(
    robot='turtlebot3',
    world='turtlebot3_world',
    headless=False
)

print("Gazebo launched with TurtleBot3")
print("Environment: turtlebot3_world (known map)")
time.sleep(3)

## 2. Monte Carlo Localization Setup

### Motion Model
Odometry-based motion model with noise:
$$\mathbf{x}_k = \mathbf{x}_{k-1} + \mathbf{u}_k + \mathbf{w}_k$$

where:
- $\mathbf{x} = [x, y, \theta]$
- $\mathbf{u}$ = odometry motion
- $\mathbf{w}$ = motion noise

### Measurement Model
LIDAR likelihood based on expected vs measured ranges:
$$p(\mathbf{z}_k | \mathbf{x}_k) \propto \exp\left(-\sum_i \frac{(z_i - \hat{z}_i(\mathbf{x}_k))^2}{2\sigma^2}\right)$$

In [ ]:
# Map bounds (turtlebot3_world is approximately 8m x 8m)
MAP_BOUNDS = {
    'x_min': -4.0,
    'x_max': 4.0,
    'y_min': -4.0,
    'y_max': 4.0
}

# Particle filter parameters
N_PARTICLES = 1000
MOTION_NOISE = np.array([0.02, 0.02, 0.05])  # [x, y, theta]
MEASUREMENT_NOISE = 0.2  # LIDAR noise std dev (m)

print(f"MCL Parameters:")
print(f"  Particles: {N_PARTICLES}")
print(f"  Map bounds: {MAP_BOUNDS}")
print(f"  Motion noise: {MOTION_NOISE}")

## 3. Initialize Particles

For global localization, initialize particles uniformly across the map.

In [ ]:
def initialize_particles_uniform(n_particles, bounds):
    """
    Initialize particles uniformly across map.
    
    For kidnapped robot problem, we have no prior on location.
    """
    particles = np.zeros((n_particles, 3))
    
    # Uniform position
    particles[:, 0] = np.random.uniform(bounds['x_min'], bounds['x_max'], n_particles)
    particles[:, 1] = np.random.uniform(bounds['y_min'], bounds['y_max'], n_particles)
    
    # Uniform orientation
    particles[:, 2] = np.random.uniform(-np.pi, np.pi, n_particles)
    
    weights = np.ones(n_particles) / n_particles
    
    return particles, weights

# Initialize
particles_init, weights_init = initialize_particles_uniform(N_PARTICLES, MAP_BOUNDS)

print(f"\nInitialized {N_PARTICLES} particles uniformly across map")
print(f"Mean initial position: ({particles_init[:, 0].mean():.2f}, {particles_init[:, 1].mean():.2f})")

## 4. Motion and Measurement Models

In [ ]:
def motion_model(particle, odom_delta):
    """
    Apply odometry motion to particle with noise.
    
    Parameters
    ----------
    particle : np.ndarray
        Current particle [x, y, theta]
    odom_delta : np.ndarray
        Odometry change [dx, dy, dtheta]
    
    Returns
    -------
    np.ndarray
        Updated particle
    """
    x, y, theta = particle
    dx, dy, dtheta = odom_delta
    
    # Add motion noise
    noise = np.random.normal(0, MOTION_NOISE)
    
    # Rotate motion into particle's frame
    dx_global = dx * np.cos(theta) - dy * np.sin(theta)
    dy_global = dx * np.sin(theta) + dy * np.cos(theta)
    
    # Update pose
    x_new = x + dx_global + noise[0]
    y_new = y + dy_global + noise[1]
    theta_new = theta + dtheta + noise[2]
    
    # Wrap angle
    theta_new = np.arctan2(np.sin(theta_new), np.cos(theta_new))
    
    return np.array([x_new, y_new, theta_new])

def lidar_likelihood(scan_ranges, particle, expected_ranges_func):
    """
    Compute likelihood of LIDAR scan given particle pose.
    
    Simplified model: compare measured vs expected ranges.
    """
    # Get expected ranges for this particle
    expected_ranges = expected_ranges_func(particle)
    
    # Compute likelihood (Gaussian)
    diff = scan_ranges - expected_ranges
    likelihood = np.exp(-0.5 * np.sum(diff**2) / MEASUREMENT_NOISE**2)
    
    return likelihood + 1e-10  # Avoid zero

print("Motion and measurement models defined")

## 5. Simplified Map Model

For this demonstration, we'll use a simplified map model. In practice, use a full occupancy grid or ray-casting.

In [ ]:
def expected_lidar_ranges_simple(particle, n_beams=8):
    """
    Simplified expected LIDAR ranges based on map walls.
    
    Assumes rectangular room with walls at map boundaries.
    """
    x, y, theta = particle
    ranges = []
    
    for i in range(n_beams):
        # Beam angle
        angle = theta + (i / n_beams) * 2 * np.pi
        
        # Ray direction
        dx = np.cos(angle)
        dy = np.sin(angle)
        
        # Find intersection with walls
        if abs(dx) > 1e-6:
            if dx > 0:
                range_x = (MAP_BOUNDS['x_max'] - x) / dx
            else:
                range_x = (MAP_BOUNDS['x_min'] - x) / dx
        else:
            range_x = np.inf
        
        if abs(dy) > 1e-6:
            if dy > 0:
                range_y = (MAP_BOUNDS['y_max'] - y) / dy
            else:
                range_y = (MAP_BOUNDS['y_min'] - y) / dy
        else:
            range_y = np.inf
        
        # Closest wall
        range_val = min(range_x, range_y)
        ranges.append(max(0.1, min(range_val, 10.0)))  # Clamp to sensor limits
    
    return np.array(ranges)

print("Simplified map model defined")

## 6. MCL ROS Node

Create a ROS node for real-time Monte Carlo Localization.

In [ ]:
from geometry_msgs.msg import PoseStamped, Twist
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import Odometry

# Global state
particles_current = particles_init.copy()
weights_current = weights_init.copy()
odom_prev = None
pose_history = []

def mcl_callback(tk, laser_scan, odom):
    """
    Monte Carlo Localization callback.
    
    Parameters
    ----------
    tk : float
        Current time
    laser_scan : np.ndarray
        LIDAR ranges
    odom : np.ndarray
        Odometry [x, y, theta]
    
    Returns
    -------
    dict
        Estimated pose
    """
    global particles_current, weights_current, odom_prev
    
    # Compute odometry delta
    if odom_prev is not None:
        odom_delta = odom - odom_prev
        # Wrap angle difference
        odom_delta[2] = np.arctan2(np.sin(odom_delta[2]), np.cos(odom_delta[2]))
    else:
        odom_delta = np.zeros(3)
    
    odom_prev = odom.copy()
    
    # Motion update (predict)
    for i in range(len(particles_current)):
        particles_current[i] = motion_model(particles_current[i], odom_delta)
    
    # Measurement update (weight)
    for i in range(len(particles_current)):
        expected_ranges = expected_lidar_ranges_simple(particles_current[i])
        weights_current[i] = lidar_likelihood(laser_scan, particles_current[i], 
                                              lambda p: expected_lidar_ranges_simple(p))
    
    # Normalize weights
    weights_current /= (weights_current.sum() + 1e-10)
    
    # Resample (systematic)
    N_eff = 1.0 / (np.sum(weights_current**2) + 1e-10)
    if N_eff < N_PARTICLES / 2:
        # Resample
        cumsum = np.cumsum(weights_current)
        positions = (np.arange(N_PARTICLES) + np.random.random()) / N_PARTICLES
        indices = np.searchsorted(cumsum, positions)
        particles_current = particles_current[indices]
        weights_current = np.ones(N_PARTICLES) / N_PARTICLES
    
    # Estimate pose (weighted mean)
    x_est = np.average(particles_current[:, 0], weights=weights_current)
    y_est = np.average(particles_current[:, 1], weights=weights_current)
    
    # Circular mean for angle
    theta_sin = np.average(np.sin(particles_current[:, 2]), weights=weights_current)
    theta_cos = np.average(np.cos(particles_current[:, 2]), weights=weights_current)
    theta_est = np.arctan2(theta_sin, theta_cos)
    
    pose_est = np.array([x_est, y_est, theta_est])
    pose_history.append(pose_est)
    
    return {'pose_estimate': pose_est}

print("MCL callback defined")

## 7. Create and Run MCL Node

In [ ]:
# Create MCL node
mcl_node = ROSNode(
    callback=mcl_callback,
    subscribes_to=[
        ('/scan', LaserScan, 'laser_scan'),
        ('/odom', Odometry, 'odom')
    ],
    publishes_to=[
        ('pose_estimate', PoseStamped, '/mcl/pose')
    ],
    node_name='mcl_localization',
    rate_hz=10.0
)

print("MCL node created")
print("\nSubscriptions:")
print("  - /scan (LIDAR)")
print("  - /odom (Odometry)")
print("\nPublications:")
print("  - /mcl/pose (Estimated pose)")

In [ ]:
# Start MCL node
mcl_node.create_node()
mcl_node.start()

print("\nMCL node running!")
print("Robot should be localizing...")
print("Drive the robot around to see particle convergence\n")

# Run for 30 seconds
try:
    time.sleep(30)
except KeyboardInterrupt:
    print("\nInterrupted by user")

# Stop node
mcl_node.stop()
print("\nMCL node stopped")

## 8. Visualize Results

In [ ]:
if len(pose_history) > 0:
    pose_hist_array = np.array(pose_history)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Final particle distribution
    axes[0, 0].scatter(particles_current[:, 0], particles_current[:, 1],
                      s=weights_current * 1000, alpha=0.3, c='blue')
    axes[0, 0].plot(pose_hist_array[-1, 0], pose_hist_array[-1, 1],
                   'r*', markersize=20, label='Final estimate')
    axes[0, 0].set_xlim([MAP_BOUNDS['x_min'], MAP_BOUNDS['x_max']])
    axes[0, 0].set_ylim([MAP_BOUNDS['y_min'], MAP_BOUNDS['y_max']])
    axes[0, 0].set_xlabel('X position (m)')
    axes[0, 0].set_ylabel('Y position (m)')
    axes[0, 0].set_title('Final Particle Distribution', fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    axes[0, 0].set_aspect('equal')
    
    # Trajectory estimate
    axes[0, 1].plot(pose_hist_array[:, 0], pose_hist_array[:, 1],
                   'b-', linewidth=2, label='MCL trajectory')
    axes[0, 1].plot(pose_hist_array[0, 0], pose_hist_array[0, 1],
                   'go', markersize=10, label='Start')
    axes[0, 1].plot(pose_hist_array[-1, 0], pose_hist_array[-1, 1],
                   'r*', markersize=15, label='End')
    axes[0, 1].set_xlim([MAP_BOUNDS['x_min'], MAP_BOUNDS['x_max']])
    axes[0, 1].set_ylim([MAP_BOUNDS['y_min'], MAP_BOUNDS['y_max']])
    axes[0, 1].set_xlabel('X position (m)')
    axes[0, 1].set_ylabel('Y position (m)')
    axes[0, 1].set_title('Estimated Trajectory', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    axes[0, 1].set_aspect('equal')
    
    # X position over time
    axes[1, 0].plot(pose_hist_array[:, 0], 'b-', linewidth=2)
    axes[1, 0].set_xlabel('Time step')
    axes[1, 0].set_ylabel('X position (m)')
    axes[1, 0].set_title('X Position Evolution', fontweight='bold')
    axes[1, 0].grid(True)
    
    # Y position over time
    axes[1, 1].plot(pose_hist_array[:, 1], 'r-', linewidth=2)
    axes[1, 1].set_xlabel('Time step')
    axes[1, 1].set_ylabel('Y position (m)')
    axes[1, 1].set_title('Y Position Evolution', fontweight='bold')
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nMCL tracked {len(pose_history)} poses")
    print(f"Final estimate: ({pose_hist_array[-1, 0]:.2f}, {pose_hist_array[-1, 1]:.2f}, {pose_hist_array[-1, 2]:.2f})")
else:
    print("No pose history collected")

## 9. Shutdown Gazebo

In [ ]:
stop_gazebo(gz)
print("Gazebo simulation stopped")

## Conclusion

### Key Takeaways

1. **Global Localization**: MCL solves the "kidnapped robot" problem by maintaining multiple hypotheses
2. **Particle Convergence**: Particles start uniform, converge to true pose after robot movement
3. **Measurement Importance**: LIDAR scans are crucial for disambiguating symmetric environments

### MCL vs Other Localization Methods

- **MCL (Particle Filter)**: Global localization, handles multi-modal distributions
- **EKF Localization**: Local tracking only, assumes Gaussian uncertainty
- **AMCL (Adaptive MCL)**: Dynamically adjusts particle count based on uncertainty

### Practical Tips

1. **Particle count**: More particles → better global localization, higher cost
2. **Motion noise**: Tune based on wheel slippage and surface friction
3. **Resampling**: Too frequent → particle depletion, too rare → degeneracy
4. **Map quality**: Accurate maps are essential for good likelihood models

### Next Steps

- Implement full ray-casting for accurate LIDAR likelihood
- Add adaptive resampling (AMCL)
- Test with kidnapped robot scenarios
- Compare with SLAM-based localization

### Reference

Gordon, N. J., Salmond, D. J., & Smith, A. F. M. (1993). *Novel approach to nonlinear/non-Gaussian Bayesian state estimation*. IEE Proceedings F (Radar and Signal Processing), 140(2), 107-113.